# Chapter 8: Building a Chatbot Using Attention-Based Neural Networks

## The theory of attention within neural networks

In our sequence-to-sequence model for sentence translation (with no attention implemented), we used both encoders and decoders. **The encoder obtained a hidden state from the input sentence**, which was a representation of our sentence. **The decoder then used this hidden state to perform the translation steps**. A basic graphical illustration of this is as follows:

![](graphical_repr_seq2seq_model.png)

However, **decoding over the entirety of the hidden state is not necessarily the most efficient way of using this task**. This is because the hidden state represents the entirety of the input sentence; however, in some tasks (such as predicting the next word in a sentence), **we do not need to consider the entirety of the input sentence, just the parts that are relevant to the prediction** we are trying to make. We can show that by using attention within our sequence-to-sequence neural network. We can teach our model to only look at the relevant parts of the input in order to make its prediction, resulting in a much more efficient and accurate model.

Consider the following example:

***I will be traveling to Paris, the capital city of France, on the 2nd of March. My flight leaves from London Heathrow airport and will take approximately one hour.***

Let's say that we are training a model to predict the next word in a sentence. We can first input the start of the sentence:

***The capital city of France is _____.***

We would expect our model to be able to retrieve the word Paris, in this case. If we were to use our basic sequence-to-sequence model, we would transform our entire input into a hidden state, which our model would then try to extract the relevant information out of. This includes all the extraneous information about flights. You may notice here that we only need to look at a small part of our input sentence in order to identify the relevant information required to complete our sentence:

***I will be traveling to <font color=pink>Paris, the capital city of France</font>, on the 2nd of March. My flight leaves from London Heathrow airport and will take approximately one hour.***

Therefore, if we can train our model to only use the relevant information within the input sentence, we can make more accurate and relevant predictions. We can implement attention within our networks in order to achieve this.

There are two main types of attention mechanisms that we can implement: local and global attention.

### **Comparing local and global attention**

The $2$ forms of attention that we can implement within our networks are very similar, but with subtle key differences. We will start by looking at local attention.

In **<font color=pink>local attention</font>**, our model **only looks at a few hidden states from the encoder**. For example, if we are performing a sentence translation task and we are calculating the second word in our translation, the model may wish to only look at the hidden states from the encoder related to the second word in the input sentence. This would mean that our model needs to look at the second hidden state from our encoder $(h_2)$ but maybe also the hidden state before it $(h_1)$.

In the following diagram, we can see this in practice:

![](local_attention_model.png)

We first **start by calculating the aligned position, $p_t$, from our final hidden state, $h_n$**. This tells us **which hidden states we need to be looking at to make our prediction**. We then **calculate our local weights and apply them to our hidden states in order to determine our context vector**. These weights may tell us **to pay more attention to the most relevant hidden state $(h_2)$ but less attention to the preceding hidden state $(h_1)$**.

We then **take our context vector and pass it forward to our decoder in order to make its prediction.** In our **non-attention based sequence-to-sequence model, we would have only passed our final hidden state, $h_n$, forward**, but we see here that instead, we **only consider the relevant hidden states that our model deems necessary** to make its prediction.

The **<font color=pink>global attention</font>** model **works in a very similar way**. **However, instead of only looking at a few of the hidden states, we want to look at all of our model's hidden states**—hence the name global. We can see a graphical illustration of a global attention layer here:

![](global_attention_model.png)

We can see in the preceding diagram that although this appears very similar to our local attention framework, **our model is now looking at all the hidden states and calculating the global weights across all of them**. This **allows our model to look at any given part of the input sentence that it considers relevant, instead of being limited to a local area determined by the local attention methodology**. Our model may wish to only look at a small, local area, but this is within the capabilities of the model. **An easy way to think of the global attention framework is that it is essentially learning a mask that only allows through hidden states that are relevant to our prediction:**

![](combined_model.png)

We can see in the preceding diagram that **by learning which hidden states to pay attention to, our model controls which states are used in the decoding step to determine our predicted output**. Once we decide which hidden states to pay attention to, we can combine them using a number of different methods—either by concatenating or taking the weighted dot product.

## Building a chatbot using sequence-to-sequence neural networks with attention

### **Acquiring our dataset**

### **Processing our dataset**

In [40]:
# 1 Let's read the dataset
import os
corpus = "movie_corpus"
corpus_name = "movie_corpus"
datafile = os.path.join(corpus, "formatted_movie_lines.txt")
with open(datafile, 'rb') as file:
    lines = file.readlines()
    for line in lines[:3]:
        print(str(line) + '\n')

b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\n"

b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\n"

b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"



### **Creating the vocabulary**

In the past, our corpus has comprised of several dictionaries consisting of the unique words in our corpus and lookups between word and indices. However, we can do this in a far more elegant way **by creating a vocabulary class that consists of all of the elements required**:

1. We start by creating our `vocabulary` class. We initialize this class with empty dictionaries-`word2index` and `word2count`. We also initialize this `index2word` dictionary with placeholders for out padding tokens, as well as our `Start-of-Sentence (SOS)` and `End-of-Sentence (EOS)` tokens. We keep a running count of the number of words in our vocabulary, too (which is $3$ to start with as our corpus already contains the three tokens mentioned). These are the default values for an empty vocabulary; however, they will be populated as we read our data in:

```python
PAD_token = 0
SOS_token = 1
EOS_token = 2

class Vocabularity:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3
```

2. Next, we create the functions that we create the functions that we will use to populate our vocabulary. `addWord` takes a word as input. If this is a new word that is not already in our vocabulary, we add this word to our indices, set the count of this word to $1$, and increment the total number of words in our vocabulary by $1$. If the word in question is already in our vocabulary, we simply increment the count of this word by $1$.

```python
def addWord(self, w):
    if w not in self.word2index:
        self.word2index[w] = self.num_words
        self.word2count[w] = 1
        self.index2word[self.num_words] = w
        self.num_words += 1
    else:
        self.word2count[w] += 1
```

3. We also use the `addSentence` function to apply the `addWord` function to all the words within a given sentence:

```python
def addSentence(self, sent):
    for word in sent.split(''):
        self.addWord(word)
```

One thing we can do to speed up the training of our model is reduce the size of our vocabulary. This means that any embedding layers will be much smaller and the total number of learned parameters within our model can be fewer. An easy way to do this is to remove any low-frequency words from our vocabulary. Any words occurring just once or twice in our dataset are unlikely to have huge predictive power, and so removing them from our corpus and replacing them with blank tokens in our final model could reduce the time taken for our model to train and reduce overfitting without having much of a negative impact on our model's predictions.

4. To remove low-frequency words from our vocabulary, we can implement a trim function. The function first loops through the word count dictionary and if the occurrence of the word is greater than the minimum required count, it is appended to a new list:

```python
def trim(self, min_cnt):
    if self.trimmed:
        return
    self.trimmed = True
    words_to_keep = []
    for k in self.word2count.items():
        if v >= min_cnt:
            words_to_keep.append(k)
    print('Words to keep: {} / {} = {:.2%}'.format(len(words_to_keep), len(self.word2index), len(words_to_keep)/len(words_to_keep)))
```
5. Finally, our indices are rebuilt from the `new_to_keep` list. We set all the indice to their initial empty values and then repopulate them by looping through our kept words with the `addWord` function:

```python
self.word2index = {}
self.word2count = {}
self.index2word = {PAD_token: "PAD", sos_token: "SOS", EOS_token: "EOS"}

self.num_words = 3
for w in words_to_keep:
    self.addWord(w)
```

We have defined a vocabulary class that can be easily poplulated with our input sentences, Next, we actually need to load in our dataset to create our training data.

***Here the complete code of the vocabulary class***:

In [48]:
PAD_token = 0
SOS_token = 1
EOS_token = 2

class Vocabularity:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3
    
    # 2
    def addWord(self, w):
        if w not in self.word2index:
            self.word2index[w] = self.num_words
            self.word2count[w] = 1
            self.index2word[self.num_words] = w
            self.num_words += 1
        else:
            self.word2count[w] += 1

    # 3
    def addSentence(self, sent):
        for word in sent.split(' '):
            self.addWord(word)

    # 4
    def trim(self, min_cnt):
        if self.trimmed:
            return
        self.trimmed = True
        words_to_keep = []
        for k in self.word2count.items():
            if k >= min_cnt:
                words_to_keep.append(k)
        print('Words to keep: {} / {} = {:.2%}'.format(len(words_to_keep), len(self.word2index), len(words_to_keep)/len(words_to_keep)))
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}

        self.num_words = 3
        for w in words_to_keep:
            self.addWord(w)

### **Loading the data**

We will start loading in the data using the following steps:

In [42]:
# 1 The first step for reading in our data is to perform any
# necessary steps to clean the data and make it more
# human-readable. We start by converting it from Unicode
# into ASCII format. We can easily use a function to do this:
import unicodedata
import re

def unicodeToAscii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

# 2 Next, we want to process our input strings so that they are
# all in lowercase and do not contain any trailing whitespace or
# punctuation, except the most basic characters. We can do this
# by using a series of regular expressions
def cleanStrings(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# 3 Finally, we apply this function within a wider funtion
# readVocs. This functions reads our data file into lines and
# then applies the cleanString function to every line. It also
# creates an instance of the vocabulary class that we created earlier
# meaning, this function outputs both our data data and vocabulary:
def readVocs(datafile, corpus_name):
    lines = open(datafile, encoding='utf-8').read().strip().split('\n')
    pairs = [[cleanStrings(s) for s in l.split('\t')] for l in lines]
    voc = Vocabularity(corpus_name)
    return voc, pairs

Next, we filter our input pairs by their maximum length. This is again done to reduce the potential dimensionality of our model. Predicting sentences that are hundreds of words long would require a very deep architecture. In the interest of training time, we want to limit our training data here to instances where the input and output are less than $10$ words long.

4. To do this, we create a couple of filter functions. The first one, `filterPair`, returns a Boolean value based on whether the current line has an input and output length that is less than the maximum length. Our second function, `filterPairs`, simply applies this condition to all the pairs within our dataset, only keeping the ones that meet this condition:

In [43]:
def filterPair(p, max_length):
    return len(p[0].split(' ')) < max_length and len(p[1].split(' ')) < max_length

def filterPairs(pairs, max_length):
    return [pair for pair in pairs if filterPair(pair, max_length)]

5. Now, we just need to create one final function that applies all the previous functions we have put together and run it to create our vocabulary and data pairs:

In [44]:
def loadData(corpus, corpus_name, datafile, max_length):
    voc, pairs = readVocs(datafile, corpus_name)
    print(str(len(pairs)) + " Sentence pairs")
    pairs = filterPairs(pairs, max_length)
    print(str(len(pairs)) + " Sentence pairs after trimming")
    for p in pairs:
        voc.addSentence(p[0])
        voc.addSentence(p[1])
    print(str(voc.num_words) + " Distinct words in vocabulary")
    return voc, pairs

max_length = 10
voc, pairs = loadData(corpus, corpus_name, datafile, max_length)

221282 Sentence pairs
70015 Sentence pairs after trimming
27501 Distinct words in vocabulary


We can see that our input dataset consists of over $200,000$ pairs. When we filter this to sentences where both the input and output are less than $10$ words long, this reduces to just $64,000$ pairs consisting of $18,000$ distinct word (previous result):

In [55]:
#  6 We can print a selection of our processed input/output
# pairs in order to verify that our functions have all worked correctly:
print("Example Pairs:")
for pair in pairs[-10:]:
    print(pair)

Example Pairs:
['three minutes to go !', 'yes .']
['yes .', "what d'ya want to do to kill time ?"]
['another fifteen seconds to go .', 'do something ! stall them !']
['yes, sir, name, please ?', 'food !']
['food !', 'do you have a reservation ?']
['do you have a reservation ?', 'food ! !']
['grrrhmmnnnjkjmmmnn !', 'franz ! help ! lunatic !']
["what o'clock is it, mr noggs ?", "eleven o'clock, my lorj 42"]
['stuart ?', 'yes .']
['yes .', 'how quickly can you move your artillery forward ?']


It appears that we have successfully split our dataset into input and output pairs upon which we can train our network.

Finally, before we begin building the model, we must remove the rare words from our corpus and data pairs.

### **Removing rare words**

As previously mentioned, including words that only occur a few times within our dataset will increase the dimensionality of our model, increasing our model's complexity and the time it will take to train the model. Therefore, it is preferred to remove them from our training data to keep our model as streamlined and efficient as possible.

You may recall earlier that we built a trim function into our vocabulary, which will allow us to remove infrequently occurring words from our vocabulary. We can now create a function to remove these rare words and call the trim method from our vocabulary as our first step. You will see that this removes a large percentage of words from our vocabulary, indicating that the majority of the words in our vocabulary occur infrequently. This is expected as the distribution of words within any language model will follow a long-tail distribution. We will use the following steps to remove the words:

In [56]:
# 1 We first calculate the percentage of words that we will keep within our model
def removeRareWords(voc, all_pairs, minimum):
    voc.trim(minimum)
    # 2 we loop through all the words in the input and output sentences.
    # If for a given pair either the input or output sentence has a word
    # that isn't in our new trimmed corpus, we drop this pair from our dataset.
    # We print the output and see that even though we have dropped over half
    # of our vocabulary, we only drop around 17% of our training pairs. This
    # again reflects how our corpus of words is distributed over our individual
    # training pairs:
    pairs_to_keep = []
    for p in all_pairs:
        keep = True
        for word in p[0].split(' '):
            if word not in voc.word2index:
                keep = False
                break
        for word in p[1].split(' '):
            if word not in voc.word2index:
                keep = False
                break
        if keep:
            pairs_to_keep.append(p)


    print("Trimmed from {} pairs to {}, {:.2%} of total".format(len(all_pairs), len(pairs_to_keep), len(pairs_to_keep)/ len(all_pairs)))
    return pairs_to_keep
minimum_count = 3
pairs = removeRareWords(voc, pairs, minimum_count)


Trimmed from 70015 pairs to 70015, 100.00% of total


### **Transforming sentence pairs to tensors**

We know that **our model will not take raw text as input**, but rather, tensor representations of sentences. We will also **not process our sentences one by one, but instead in smaller batches**. For this, **we require both our input and output sentences to be transformed into tensors**, where the width of the tensor represents the size of the batch that we wish to train on:

In [57]:
import itertools
import random
import torch
# 1 We start by creating helper functions, which we can use to transform
# out pairs into tensors. We first create a indexFromSentence function,
# which grabs the index of each word in the sentence from the vocabulart and
# appends EOS token to the end
def indexFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ') + [EOS_token]]

# 2 Secondly, we create a zeroPad function which pads any tensors with
# zeros so that all of the sentences within the tensor are effectively the
# same length
def zeroPad(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))
# 3 Then, to generate our input tensor, we apply both of these functions
#  First, we get the indices of our input sentence, then apply padding,
# and then transform the output into LongTensor. We will also obtain the
# lengths of each of our input sentences out output this as a tensor:
def inputVar(l, voc):
    indexes_batch = [indexFromSentence(voc, sentence) for sentence in l]
    padList = zeroPad(indexes_batch)
    padTensor = torch.LongTensor(padList)
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    return padTensor, lengths

# 4 Within our model, our padded tokens should gererally be ignored.
# We don't want to train our model on these padded tokens, so we create a
# boolean mask to ignore these tokens. To do so, we use a getMask function,
# which we apply to our output tensor. This simply returns 1 if the output consists
# of a word and 0 if it consists of a padding token
def getMask(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# 5 We then apply this to our outputVar function, this is identical to the inputVar
# function except that along with the indexed output tensor and the tensor of lengths,
# we also return the boolean mask of our output tensor. This boolean mask just returns
# True when there is a word within the output tensor and False when there is a padding
# token. We also return the maximum length of sentences within our output tensor:
def outputVar(l, voc):
    indexes_batch = [indexFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPad(indexes_batch)
    mask = torch.BoolTensor(getMask(padList))
    padTensor = torch.LongTensor(padList)
    return padTensor, mask, max_target_len

# 6 Finally in order to create our input and output batches concurrently, we loop
# through the pairs in our batch and create input and output tensors for both pairs
# using the functions we created previously. We then return all the necessary variables:
def batch2Train(voc, batch):
    batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch = []
    output_batch = []

    for p in batch:
        input_batch.append(p[0])
        output_batch.append(p[1])
        inp, lengths = inputVar(input_batch, voc)
        output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len

# 7 This function should be all we need to transform our training pairs into
# tensors for training our model. We can validate that this is working correctly
# by performing a single iteration of our batch2Train function on a random
# selection of our data. We set our batch size to 5 and run this once:

test_batch_size = 5

batches = batch2Train(voc, [random.choice(pairs) for _ in range(test_batch_size)])

input_variable, lengths, target_variable, mask, max_target_len = batches

KeyError: 2

## Constructing the model